In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as tf
from tensorflow.keras.layers import Dense,LSTM,Conv2D,MaxPooling2D,InputLayer,BatchNormalization,Dropout,Bidirectional
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data=pd.read_csv("/kaggle/input/playground-series-s4e12/train.csv")
data.head(10)

In [ ]:
data=data.dropna()
data.head(10)
data.shape

In [ ]:
print(data.columns)
len(data.dtypes)


In [ ]:
data["Policy Start Date"]=pd.to_datetime(data["Policy Start Date"])

data["year"]=data["Policy Start Date"].dt.year

data["month"]=data["Policy Start Date"].dt.month

data["day"]=data["Policy Start Date"].dt.day

data=data.drop("Policy Start Date",axis=1)

In [ ]:
print(data["Location"][:5])

In [ ]:
object_column_names = data.select_dtypes(include='object').columns

print(object_column_names)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
hottie=LabelEncoder()

data_to_encode=['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location', 'Policy Type', 'Customer Feedback', 'Smoking Status', 'Exercise Frequency', 'Property Type']

encoded_data=data.apply(lambda col: hottie.fit_transform(col) if col.dtype == "object" else col)



In [ ]:
data=data.drop(['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location', 'Policy Type', 'Customer Feedback', 'Smoking Status', 'Exercise Frequency', 'Property Type'],axis=1)

In [ ]:
data=encoded_data

In [ ]:
datay=data["Premium Amount"]
data=data.drop("Premium Amount",axis=1)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

hello1=StandardScaler()
data=hello1.fit_transform(data)

hello = PCA(n_components=18)
x_data=hello.fit_transform(data)

x_data=pd.DataFrame(x_data)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr = x_data.corr()

sns.heatmap(corr,annot=False,cmap="coolwarm", annot_kws={"size": 8} ,fmt=".2f")
plt.title("shows that the correlation is independent and the pca is perfect")

In [ ]:
data_x=np.reshape(x_data,(x_data.shape[0],18,1))

datay = np.reshape(datay, (-1, 1))


In [ ]:
model=Sequential(
    [
        Bidirectional(LSTM(128,activation="relu",input_shape=(18,1),return_sequences=True)),
        Bidirectional((128,activation="relu",return_sequences=True)),
        Dropout(0.2),
        BatchNormalization(),
        Birectional(LSTM(64,activation="relu",return_sequences=True)),
        BatchNormalization(),
        Dropout(0.2),
        Bidirectional(LSTM(32,activation="relu",return_sequences=False)),
        Dense(32,activation="relu",),
        Dense(1,activation="linear")
    ]
)

In [ ]:
model.compile(optimizer="adam",metrics=["MAE"],loss="MAE")

In [ ]:
model.fit(x_data,datay,epochs=20)

In [ ]:
model.save("model.h5")